In [4]:
import pandas as pd
import numpy as np
import gzip
from tqdm import tnrange, tqdm_notebook
import scipy
import operator
import difflib
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
    
 
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [20]:
path_merge_linux='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Merged_LinkedIn_WoS/'
# #file_pickle = open(path_merge_cluster+"list_relevant_papers.pickle",'rb')        ### num papers:  2674932
# file_pickle = open(path_merge_linux+"list_relevant_papers_partial.pickle",'rb')        ### num papers:  2674932
# list_involved_papers  = pickle.load(file_pickle)
# print "# papers involved:   ",len(list_involved_papers)



file_pickle = open(path_merge_linux+"list_involved_papers_all_lines.pickle",'rb')
list_involved_papers  = pickle.load(file_pickle)
set_list_involved_papers=set(list_involved_papers)
print len(list_involved_papers)


2674932


In [24]:
path_cluster="/webofscience/diego/WoS_tables/references/paper_ID_ref_IDs/"
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/WoS_data/Citation_data/"


lista_files=["data_references_IDs_2013.txt.gz","data_references_IDs_2014.txt.gz","data_references_IDs_2015.txt.gz"] 




# lista_files=["data_references_IDs_1963.txt.gz","data_references_IDs_1977.txt.gz","data_references_IDs_1991.txt.gz","data_references_IDs_2005.txt.gz",\
#     "data_references_IDs_1950.txt.gz","data_references_IDs_1964.txt.gz","data_references_IDs_1978.txt.gz","data_references_IDs_1992.txt.gz","data_references_IDs_2006.txt.gz",\
#     "data_references_IDs_1951.txt.gz","data_references_IDs_1965.txt.gz","data_references_IDs_1979.txt.gz","data_references_IDs_1993.txt.gz","data_references_IDs_2007.txt.gz",\
#     "data_references_IDs_1952.txt.gz","data_references_IDs_1966.txt.gz","data_references_IDs_1980.txt.gz","data_references_IDs_1994.txt.gz","data_references_IDs_2008.txt.gz",\
#     "data_references_IDs_1953.txt.gz","data_references_IDs_1967.txt.gz","data_references_IDs_1981.txt.gz","data_references_IDs_1995.txt.gz","data_references_IDs_2009.txt.gz",\
#     "data_references_IDs_1954.txt.gz","data_references_IDs_1968.txt.gz","data_references_IDs_1982.txt.gz","data_references_IDs_1996.txt.gz","data_references_IDs_2010.txt.gz",\
#     "data_references_IDs_1955.txt.gz","data_references_IDs_1969.txt.gz","data_references_IDs_1983.txt.gz","data_references_IDs_1997.txt.gz","data_references_IDs_2011.txt.gz",\
#     "data_references_IDs_1956.txt.gz","data_references_IDs_1970.txt.gz","data_references_IDs_1984.txt.gz","data_references_IDs_1998.txt.gz","data_references_IDs_2012.txt.gz",\
#     "data_references_IDs_1957.txt.gz","data_references_IDs_1971.txt.gz","data_references_IDs_1985.txt.gz","data_references_IDs_1999.txt.gz","data_references_IDs_2013.txt.gz",\
#     "data_references_IDs_1958.txt.gz","data_references_IDs_1972.txt.gz","data_references_IDs_1986.txt.gz","data_references_IDs_2000.txt.gz","data_references_IDs_2014.txt.gz",\
#     "data_references_IDs_1959.txt.gz","data_references_IDs_1973.txt.gz","data_references_IDs_1987.txt.gz","data_references_IDs_2001.txt.gz","data_references_IDs_2015.txt.gz",\
#     "data_references_IDs_1960.txt.gz","data_references_IDs_1974.txt.gz","data_references_IDs_1988.txt.gz","data_references_IDs_2002.txt.gz",\
#     "data_references_IDs_1961.txt.gz","data_references_IDs_1975.txt.gz","data_references_IDs_1989.txt.gz","data_references_IDs_2003.txt.gz",\
#     "data_references_IDs_1962.txt.gz","data_references_IDs_1976.txt.gz","data_references_IDs_1990.txt.gz","data_references_IDs_2004.txt.gz"] 


In [17]:
#%timeit set_involved_papers = set(list_involved_papers)  muy lento de hacer!!!
test = ['a','b','c']
# probar:    [paper for paper in list_ref if paper in set_involved_papers] en lugar de la interseccion

['None']

In [25]:


test_lines=5000000000


string_lines="_all_lines"
if test_lines < 1000000000:
     string_lines="_"+str(test_lines)+"lines"

        
tot_cont=0
tot_cont_useful_lines=0
tot_cont_useful_lines_adding_cit =0       

dict_paper_id_list_papers_that_cite_it={}          

   
    
    
        #file_citations="data_references_IDs_1950.txt.gz"   # wc -l         1095263  in 2008              1471208  in 2014  (about the largest file of all)
        
        
    # example:    WOS:000208613900218||WOS:000171098400003;WOS:000078872600001;WOS:000089084800005;WOS:000172993900077;WOS:000224947200006;WOS:000232448400002;000208613900218.2;000208613900218.7;000208613900218.8||
    # paper ||  ref1; ref2;...;refn||
      # with a ref has the form:  000208613900218.2  means that the focus paper (000208613900218) is making a ref. to a paper NOT included in WOS


for file_citations in  tqdm_notebook(sorted(lista_files)):
 
    print "reading:", file_citations," ........."

    cont=0       
    cont_useful_lines_adding_cit =0
    with gzip.open(path+file_citations,'r') as f:        

         for line in f:            
            cont +=1            
            tot_cont +=1

            if cont == test_lines:
                break


            lista_line=line.split("||")
            paper=lista_line[0].replace("WOS:","")   # papers are str            
            list_ref=lista_line[1].replace("WOS:","").split(";")

            ref_overlap=list(set(list_ref) & set_list_involved_papers)
            if   len(ref_overlap)>0:

                cont_useful_lines_adding_cit +=1
                tot_cont_useful_lines_adding_cit +=1

                for ref in ref_overlap:
                    try:
                        dict_paper_id_list_papers_that_cite_it[ref].append(paper)
                    except KeyError:
                        dict_paper_id_list_papers_that_cite_it[ref]=[]
                        dict_paper_id_list_papers_that_cite_it[ref].append(paper)

                

    print  "num lines read:", cont, "   with useful info:",cont_useful_lines_adding_cit

    ####### i dump the dict after reading every year's file
    filename_pickle=path_merge_linux+"dict_paper_id_list_papers_that_cite_it"+string_lines+".pickle"      
    pickle.dump(dict_paper_id_list_papers_that_cite_it, open(filename_pickle, 'wb'))
    print "written:",filename_pickle  




print "done!"
print "num lines read:", tot_cont, "   with useful info:",tot_cont_useful_lines_adding_cit


reading: data_references_IDs_1950.txt.gz  .........



IOError: [Errno 2] No such file or directory: '/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/WoS_data/Citation_data/data_references_IDs_1950.txt.gz'

In [ ]:
dict_paper_id_list_papers_that_cite_it

{'A1990DL50500019': ['A1993LC08200038'],
 'A1980KB41800016': ['000171935500009'],
 'A1992HK69600006': ['000304983300001'],
 'A1995QE62800003': ['000075256200016'],
 '000077836600013': ['000241477400081'],
 'A1991GW84500006': ['A1993LG89200001', '000346258400001'],
 'A1957CCW3400002': ['A1973R293000007'],
 '000082676300020': ['000327814600015'],
 '000264524300011': ['000341480000004'],
 'A1987F852600037': ['A1988P380200043', 'A1989U275400069', 'A1990DF98000024'],
 'A1985ADT9900055': ['A1988P380200043'],
 '000263572100015': ['000291227700012'],
 'A1995QH04200030': ['000182794700004', '000260570200009'],
 'A1991HB56300019': ['A1993MF32300023'],
 'A1995RY33200015': ['000084940100020'],
 'A1985AUE4700012': ['A1986C266700002'],
 'A1990DE08100013': ['000231650700007'],
 'A1992JX29500052': ['A1994NY00200048'],
 'A1984TV78700005': ['A1988R721400004'],
 '000072588400014': ['000174634200027'],
 'A1993LY32500019': ['000087256100001'],
 'A1994NA02700003': ['000084875900009'],
 '000233429400008': ['

{'A1990DL50500019': ['A1993LC08200038'],
 'A1980KB41800016': ['000171935500009'],
 'A1992HK69600006': ['000304983300001'],
 'A1995QE62800003': ['000075256200016'],
 '000077836600013': ['000241477400081'],
 'A1991GW84500006': ['A1993LG89200001', '000346258400001'],
 'A1957CCW3400002': ['A1973R293000007'],
 '000082676300020': ['000327814600015'],
 '000264524300011': ['000341480000004'],
 'A1987F852600037': ['A1988P380200043', 'A1989U275400069', 'A1990DF98000024'],
 'A1985ADT9900055': ['A1988P380200043'],
 '000263572100015': ['000291227700012'],
 'A1995QH04200030': ['000182794700004', '000260570200009'],
 'A1991HB56300019': ['A1993MF32300023'],
 'A1995RY33200015': ['000084940100020'],
 'A1985AUE4700012': ['A1986C266700002'],
 'A1990DE08100013': ['000231650700007'],
 'A1992JX29500052': ['A1994NY00200048'],
 'A1984TV78700005': ['A1988R721400004'],
 '000072588400014': ['000174634200027'],
 'A1993LY32500019': ['000087256100001'],
 'A1994NA02700003': ['000084875900009'],
 '000233429400008': ['